# NYC Taxi Fare & Duration
## Model Training

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wget

sys.path.append("..")

import source.configs as configs
import source.preprocessing as preprocessing

# Check if we are in COLAB
IN_COLAB = 'google.colab' in sys.modules

In [15]:
if not os.path.exists("../dataset/yellow_tripdata_2022-05.parquet"):
    if IN_COLAB:
        !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet "../dataset"
    else:
        wget.download(configs.START_DATASET_URL, "../dataset")

In [16]:
dataset = pd.read_parquet("../dataset/yellow_tripdata_2022-05.parquet")

In [17]:
dataset = preprocessing.add_trip_duration_feature(dataset)
dataset = preprocessing.fill_na_values(dataset)
dataset = preprocessing.filter_outliers(dataset)

In [18]:
dataset = preprocessing.add_day_of_week(dataset)
dataset = preprocessing.add_hour_of_day(dataset)
dataset = preprocessing.map_hour_zone(dataset)
dataset = preprocessing.map_rush_hour(dataset)

In [19]:
dataset = preprocessing.create_one_hot_encodings(dataset)

### Features selection
* trip_distance
* vendor_id
* passenger_count
* day_of_week (engineered)
* hour_zone (engineered)
* rush_hour

### Targets 
* trip_duration (engineered)
* fare_amount

### Dropped
* tpep_pickup_datetime: We use engineered features extracted from this timestamp.
* tpep_dropoff_datetime: We can't use this, it's from the future.
* store_and_fwd_flag: Trip record was held in vehicle memory.
* extra: Rush hour + overnight.
* mta_tax: Autmatic triggered based on rate in use.
* improvement_surcharge: Surcharge.
* total_amount: Fare + taxes + etc.
* tip_amount: Not used.
* congestion_surcharge: Not included for now.
* PULocationID & DOLocationID: Taxi zones, add 100s of categorical features, not used.
* payment_type: Should not be relevant.
* tolls_amount: Not used.
* airport_fee: Not used.

In [20]:
features_drop_list = ["tpep_pickup_datetime",
                    "tpep_dropoff_datetime", 
                    "store_and_fwd_flag", 
                    "extra", 
                    "mta_tax", 
                    "improvement_surcharge", 
                    "total_amount",
                    "tip_amount" ,
                    "congestion_surcharge", 
                    "PULocationID",
                    "DOLocationID",
                    "payment_type",
                    "tolls_amount",
                    "airport_fee"]

features_drop_list.append("hour_of_day")

In [21]:
dataset = preprocessing.drop_features(dataset, features_drop_list)

In [22]:
dataset.columns.to_list()

['passenger_count',
 'trip_distance',
 'RatecodeID',
 'fare_amount',
 'trip_duration',
 'day_of_week',
 'rush_hour',
 'RatecodeID_1.0',
 'RatecodeID_2.0',
 'RatecodeID_3.0',
 'RatecodeID_4.0',
 'RatecodeID_5.0',
 'RatecodeID_6.0',
 'VendorID_1',
 'VendorID_2',
 'VendorID_5',
 'VendorID_6',
 'day_of_week_0',
 'day_of_week_1',
 'day_of_week_2',
 'day_of_week_3',
 'day_of_week_4',
 'day_of_week_5',
 'day_of_week_6',
 'hour_zone_Afternoon',
 'hour_zone_Evening',
 'hour_zone_Morning',
 'hour_zone_Night',
 'hour_zone_Noon']

In [23]:
for col in dataset.isna().sum():
    assert col==0

In [24]:
dataset.describe()

,passenger_count,trip_distance,RatecodeID,fare_amount,trip_duration,day_of_week,rush_hour,RatecodeID_1.0,RatecodeID_2.0,RatecodeID_3.0,...,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,hour_zone_Afternoon,hour_zone_Evening,hour_zone_Morning,hour_zone_Night,hour_zone_Noon
count,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,...,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06,3.588295e+06
mean,1.341704e+00,3.400188e+00,1.083079e+00,1.253954e+01,1.822221e+01,2.918017e+00,3.176007e-01,9.465144e-01,4.103314e-02,3.242766e-03,...,1.395510e-01,1.434860e-01,1.372239e-01,1.301445e-01,1.403318e-01,1.897141e-01,2.428981e-01,2.330787e-01,1.607117e-01,1.735975e-01
std,7.744593e-01,3.996767e+00,4.307322e-01,8.072620e+00,5.156555e+01,2.000686e+00,4.655433e-01,2.249997e-01,1.983669e-01,5.685289e-02,...,3.465206e-01,3.505678e-01,3.440837e-01,3.364624e-01,3.473310e-01,3.920748e-01,4.288341e-01,4.227920e-01,3.672649e-01,3.787630e-01
min,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.150000e+00,1.000000e+00,7.500000e+00,7.666667e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.960000e+00,1.000000e+00,1.050000e+01,1.270000e+01,3.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.000000e+00,3.600000e+00,1.000000e+00,1.475000e+01,2.061667e+01,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,5.000000e+00,2.000000e+01,6.000000e+00,5.000000e+01,6.823550e+03,6.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [30]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
# Split features and targets
X, y_td, y_fa = preprocessing.split_dataset(dataset)

In [32]:
X_train, X_test, y_train_td, y_test_td = train_test_split(X, y_td, test_size=0.2, random_state=42)

In [33]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [42]:
model_td = RandomForestRegressor(n_jobs=-1, n_estimators=40)
#model_td = LinearRegression()
model_td.fit(X_train_scaled, y_train_td.values.ravel())
predictions = model_td.predict(X_test_scaled)

In [43]:
mse_td = mean_squared_error(y_test_td, predictions)
r2_td = r2_score(y_test_td, predictions)

In [44]:
print(f"MSE: {mse_td}")
print(f"re: {r2_td}")

MSE: 2802.5777391211564
re: -0.062444797793963724
